# Spectra Reparametrization

We calculate and visualize the reconstruction of stellar spectra with and without recasting to same physical parameters.

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import sys
import pickle


from tagging.src.datasets import ApogeeDataset
from tagging.src.networks import ConditioningAutoencoder,Embedding_Decoder,Feedforward
from tagging.src.utils import get_batch, invert_x,get_xdata
from tagging.paths import basepath,path_dataset


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## We define some hyperparameters

In [ ]:
n_batch = 9 
n_bins = 7751 
n_conditioned = 2

The next line is only used for getting the wavelength axis correct

In [ ]:
xdata= get_xdata()

We next load up the data. ```data_plotting``` contains the spectra we want to visualize but because we use data-driven normalization to not mess up normalization we append it to the original data```data``` 

In [ ]:
data = pd.read_pickle(path_dataset)
data_plotting = pd.read_pickle(os.path.join(os.path.split(basepath)[0],"data","processed","spectra_visualization.pd"))

overall_data = data_plotting.append(data,ignore_index=True)

In [ ]:
dataset = ApogeeDataset(overall_data,n_bins)
evaluation_loader = torch.utils.data.DataLoader(dataset = dataset[0:18],
                                     batch_size = n_batch,
                                     shuffle = False,
                                     drop_last=True)

load the precalculated residuals. Residuals can be calculated from ```02-Calculate_neural_residuals```

In [ ]:
with open("../../outputs/intermediate/residuals/fader.p","rb") as f:
    fader_res = pickle.load(f)
    

with open("../../outputs/intermediate/residuals/factor.p","rb") as f:
    factor_res = pickle.load(f)
    
    
with open("../../outputs/intermediate/residuals/poly.p","rb") as f:
    poly_res = pickle.load(f)


We define the wavelength window we will be plotting and other aesthetic parameters

In [ ]:
n_start =0 
n_end = 256

lw = 1
ls = (0, (5, 5))
i=0

linestyles = {
     'densely dotted':        (0, (1, 1)),
     'densely dashed':        (0, (5, 1)),
    }

In [ ]:
n_batch=9
x_test1,u_test1,v_test1,idx_test1 = get_batch(0,n_batch,dataset)
x_test2,u_test2,v_test2,idx_test2 = get_batch(9,n_batch,dataset)
x_test1 = invert_x(x_test1)
x_test2 = invert_x(x_test2)

In [ ]:

fig, (ax1,ax2) = plt.subplots(2,1,sharex=True,gridspec_kw={'hspace': 0, 'wspace': 0})

ax1.plot(xdata[n_start:n_end],x_test1[i].detach().cpu().numpy()[n_start:n_end],linewidth=lw,label="$x_{1}$",c="b")
ax1.plot(xdata[n_start:n_end],x_test2[i].detach().cpu().numpy()[n_start:n_end],linewidth=lw,label="$x_{2}$",c="darkorange") 


ax2.plot(xdata[n_start:n_end],fader_res[n_start:n_end],linewidth=lw,label="FaderDis",c="b",linestyle=linestyles["densely dotted"])
ax2.plot(xdata[n_start:n_end],factor_res[n_start:n_end],linewidth=lw,label="FactorDis",c="r",linestyle=linestyles["densely dashed"])
ax2.plot(xdata[n_start:n_end],poly_res[n_start:n_end],linewidth=lw,label="PolyDis",c="g")



fig.text(0.13,0.925,"Star $x_1$: Teff= {} , logg = {}, [Fe/H]= {:.2g}         Star $x_2$: Teff= {} , logg = {}, [Fe/H]= {:.2g}".format(*data_plotting["params"][i][0:3]+data_plotting["params"][i+9][0:3]),va='center', rotation='horizontal',fontsize=16)

fig.text(0.05, 0.70, 'flux', va='center', rotation='vertical',fontsize=16)
fig.text(0.05, 0.30, 'residuals', va='center', rotation='vertical',fontsize=16)

ax1.set_ylim(0.6,1.0)
ax2.set_ylim(-0.015,0.015)


fig.set_size_inches(14.5, 6.5)
plt.xlim(xdata[n_start],xdata[n_end])
plt.xlabel(r"Wavelength($\AA$)",fontsize=24)

yticks1 = ax1.yaxis.get_major_ticks()
yticks1[0].set_visible(False)

yticks2 = ax2.yaxis.get_major_ticks()
yticks2[0].set_visible(False)


ax1.legend()
ax2.legend()
plt.savefig("plotWithSwapping.pdf",format="pdf")
